# Testing

In this notebook you can explore and test the resulting database tables from our Dagster ETL process.
Here you can analyze


In [2]:
import duckdb
import polars as pl

In [3]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/storage/DSP.db")
%sql conn --alias duckdb
%sql SHOW ALL TABLES;

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Running query in 'duckdb'

database,schema,name,column_names,column_types,temporary
DSP,cleaned,cleaned_knmi_weather_data,"['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y']","['BIGINT', 'DATE', 'TINYINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT']",False
DSP,cleaned,cleaned_storm_incidents,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT']",False
DSP,joined,deployment_incident_vehicles_weather,"['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y', 'Incident_ID', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']","['BIGINT', 'DATE', 'TINYINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
DSP,joined,incident_deployments,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
DSP,joined,incident_deployments_vehicles,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
DSP,public,bag_panden,"['geometry', 'identificatie', 'rdf_seealso', 'bouwjaar', 'status', 'gebruiksdoel', 'oppervlakte_min', 'oppervlakte_max', 'aantal_verblijfsobjecten']","['VARCHAR', 'VARCHAR', 'VARCHAR', 'BIGINT', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'BIGINT']",False
DSP,public,cbs_wijken,"['geometry', 'buu

In [4]:
cbs_wijken = conn.execute(
    """
    SELECT * FROM public.cbs_wijken """
).pl()

service_areas = conn.execute(
    """
    SELECT * FROM public.service_areas """
).pl()

incident_deployments_vehicles = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles"""
).pl()


cleaned_knmi_weather_data = conn.execute(
    """
    SELECT * FROM cleaned.cleaned_knmi_weather_data
    """
).pl()


# Close the database connection
conn.close()

In [6]:
incident_deployments_vehicles.head()
cleaned_knmi_weather_data.head()

Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,Sq,Q,Dr,Rh,P,Vv,N,U,Ww,Ix,M,R,S,O,Y
i64,date,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
240,2005-01-01,1,260,40,30,60,68,null,57,0,0,0,0,10246,57,8,93,10,7,0,0,0,0,0
240,2005-01-01,2,230,30,30,60,65,null,52,0,0,0,0,10244,58,8,91,10,7,0,0,0,0,0
240,2005-01-01,3,230,40,30,50,43,null,34,0,0,0,0,10241,40,1,94,10,7,0,0,0,0,0
240,2005-01-01,4,220,40,40,50,38,null,32,0,0,0,0,10239,12,0,96,10,7,0,0,0,0,0
240,2005-01-01,5,230,40,40,50,38,null,34,0,0,0,0,10237,14,3,97,10,7,0,0,0,0,0
